In [0]:
def mount_adls(storage_account, container_name):
    # Get secrets from Azure Key Vault
    client_id = dbutils.secrets.get(scope= 'sleeper-secret-scope', key= 'sleeper-project-client-id')
    tenant_id = dbutils.secrets.get(scope= 'sleeper-secret-scope', key= 'sleeper-project-tenant-id')
    client_secret = dbutils.secrets.get(scope= 'sleeper-secret-scope', key= 'sleeper-project-client-secret')

    # Set Spark configuration
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Create mount point
    mount_point = f"/mnt/{storage_account}/{container_name}"
    
    # Check if mount point already exists
    existing_mounts = [mount.mountPoint for mount in dbutils.fs.mounts()]

    if mount_point in existing_mounts:
        # Unmount the existing mount point
        dbutils.fs.unmount(mount_point)
        print(f"Unmounted previously existing mount at {mount_point}")
    
    # Mount the storage account container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/",
        mount_point = mount_point,
        extra_configs = configs)
    
    print(f"Successfully mounted abfss://{container_name}@{storage_account}.dfs.core.windows.net/ to {mount_point}")

In [0]:
# Mounting containers
storage_account = 'sleeperprojectdl'
containers_for_project = ['raw', 'processed', 'presentation']

for container in containers_for_project:
    mount_adls(storage_account, container)

/mnt/sleeperprojectdl/raw has been unmounted.
Unmounted previously existing mount at /mnt/sleeperprojectdl/raw
Successfully mounted abfss://raw@sleeperprojectdl.dfs.core.windows.net/ to /mnt/sleeperprojectdl/raw
/mnt/sleeperprojectdl/processed has been unmounted.
Unmounted previously existing mount at /mnt/sleeperprojectdl/processed
Successfully mounted abfss://processed@sleeperprojectdl.dfs.core.windows.net/ to /mnt/sleeperprojectdl/processed
Successfully mounted abfss://presentation@sleeperprojectdl.dfs.core.windows.net/ to /mnt/sleeperprojectdl/presentation


In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/sleeperprojectdl/presentation,abfss://presentation@sleeperprojectdl.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/sleeperprojectdl/processed,abfss://processed@sleeperprojectdl.dfs.core.windows.net/,
/mnt/sleeperprojectdl/raw,abfss://raw@sleeperprojectdl.dfs.core.windows.net/,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
